In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from nect.src.model.unet import UNet
from torch.profiler import profile, record_function, ProfilerActivity
import torch_extra.nn as nn_extra
import gc

In [2]:
print(torch.cuda.device_count())
print(torch.cuda.current_device())


2
0


In [3]:
gc.collect()
torch.cuda.empty_cache()

In [4]:
torch.cuda.memory_reserved()

0

In [5]:
model = UNet3D(in_dim=1, out_dim=1, num_filters=32).to(0)

In [6]:
X_train = torch.ones(20, 1, 128, 128, 128, device=0)
Y_train = X_train

In [7]:
class MSELossMSSSIM(nn.Module):
    def __init__(self) -> None:
        super(MSELossMSSSIM, self).__init__()
        self.mse = nn.MSELoss()
        self.msssim = nn_extra.MS_SSIM(data_range=1.0, spatial_dims=3, channel=1, weights=[0.0516, 0.3295, 0.3462, 0.2727])
    
    def forward(self, input, target):
        mse = self.mse(input, target)
        msssim = self.msssim(input, target)
        loss = mse + 1 - msssim
        return loss

In [8]:
n_epochs = 10
batch_size = 1
batches_per_epoch = len(X_train) // batch_size
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = MSELossMSSSIM()


In [9]:
for epoch in range(n_epochs):
    for i in range(batches_per_epoch):
        start = i * batch_size
        # take a batch
        Xbatch = X_train[start:start+batch_size]
        ybatch = Y_train[start:start+batch_size]
        # forward pass
        y_pred = model(Xbatch)
        loss = loss_fn(y_pred, ybatch)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        print(loss.item())
        print(i)
        # update weights
        optimizer.step()


1.6979339122772217
0
0.9961196780204773
1
0.9529314637184143
2
0.8756102919578552
3
0.8552221059799194
4
0.8386255502700806
5
0.8280497193336487
6
0.8208365440368652
7
0.8156527876853943
8
0.8120678067207336
9
0.8094010949134827
10
0.8063955903053284
11
0.8029520511627197
12
0.7996789216995239
13
0.7969745993614197
14
0.7945669293403625
15
0.79215407371521
16
0.7895438075065613
17
0.7865811586380005
18
0.7831820249557495
19
0.7793208360671997
0
0.7753123641014099
1
0.7723485827445984
2
0.7696572542190552
3
0.7672497630119324
4
0.7648658156394958
5
0.7626520991325378
6
0.7607199549674988
7
0.7588757276535034
8
0.7570189833641052
9
0.755145251750946
10
0.753284752368927
11
0.7514526844024658
12
0.7496365308761597
13
0.7478495240211487
14
0.7461400628089905
15
0.7444714307785034
16
0.7428178787231445
17
0.7411865592002869
18
0.7396121621131897
19
0.7380837202072144
0
0.7366008758544922
1
0.7350974082946777
2
0.7336091995239258
3
0.7321197390556335
4
0.7306572198867798
5
0.7292235493659973

KeyboardInterrupt: 